In [1]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 92 (delta 31), reused 67 (delta 15), pack-reused 0
Unpacking objects: 100% (92/92), done.


In [2]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (1030/1030), done.
remote: Compressing objects: 100% (772/772), done.
remote: Total 1030 (delta 382), reused 803 (delta 202), pack-reused 0
Receiving objects: 100% (1030/1030), 79.86 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (382/382), done.


In [3]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 91488, done.
remote: Total 91488 (delta 0), reused 0 (delta 0), pack-reused 91488
Receiving objects: 100% (91488/91488), 75.56 MiB | 27.98 MiB/s, done.
Resolving deltas: 100% (66029/66029), done.
/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 6.9 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 61 kB 643 kB/s 
     |████████████████████████████████| 895 kB 72.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Running setup.py develop for transformers
/content


In [4]:
## run_glue needs this. 
!pip install datasets

     |████████████████████████████████| 298 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.9 MB/s 
     |████████████████████████████████| 243 kB 55.4 MB/s 
     |████████████████████████████████| 132 kB 67.4 MB/s 
     |████████████████████████████████| 160 kB 72.2 MB/s 
     |████████████████████████████████| 192 kB 67.2 MB/s 
     |████████████████████████████████| 271 kB 63.1 MB/s 


In [5]:
import site
site.main()


In [6]:
import os
import csv

from pathlib import Path

In [7]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data


In [8]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


In [9]:
def insert_to_submission_file( submission_format_file, input_file, prediction_format_file, setting ) :
    submission_header, submission_content = load_csv( submission_format_file )
    input_header     , input_data         = load_csv( input_file             )
    prediction_header, prediction_data    = load_csv( prediction_format_file, '\t' )

    assert len( input_data ) == len( prediction_data )

    ## submission_header ['ID', 'Language', 'Setting', 'Label']
    ## input_header      ['label', 'sentence1' ]
    ## prediction_header ['index', 'prediction']

    prediction_data = list( reversed( prediction_data ) )

    started_insert  = False
    for elem in submission_content : 
        if elem[ submission_header.index( 'Setting' ) ] != setting :
            if started_insert :
                if len( prediction_data ) == 0 :
                    break
                else : 
                    raise Exception( "Update should to contiguous ... something wrong." ) 
            continue
        started_insert = True
        elem[ submission_header.index( 'Label' ) ] = prediction_data.pop()[ prediction_header.index( 'prediction' ) ]

    return [ submission_header ] + submission_content

In [10]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

In [11]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data


In [12]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location ) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = True,
        include_idiom   = False
    )
    write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = True,
        include_idiom    = False
    )        
    write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv',
        gold_file_name   = 'dev_gold.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

In [13]:
!ls 

AStitchInLanguageModels  SemEval_2022_Task2-idiomaticity
sample_data		 transformers


In [14]:
import pandas as pd
df = pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot.csv")
df

,DataID,Language,MWE,Setting,Previous,Target,Next,Label
0,train_zero_shot.EN.168.1,EN,double dutch,zero_shot,This inspired others to jump ropes as a leisur...,There are several theories behind the origin o...,The most popular theory states that “Double Du...,0
1,train_zero_shot.EN.168.2,EN,double dutch,zero_shot,In the age of chivalry a man paid for the woma...,"Double Dutch also derives from the same era, D...",There are many phrases that include the word: ...,0
2,train_zero_shot.EN.168.3,EN,double dutch,zero_shot,"To her eternal credit, she kept both India and...",Since 1977 we have had a plethora of Foreign M...,We need to exclude from that list the late Mr ...,0
3,train_zero_shot.EN.168.4,EN,double dutch,zero_shot,While pharmaceutical companies were researchin...,Turns out that these people were speaking doub...,So why aren’t Big Macs sold all over the world...,0
4,train_zero_shot.EN.168.5,EN,double dutch,zero_shot,Coronavirus in Europe * Brexit * Brussels ...,Is Flemish premier talking double Dutch?,Three months before the Belgians take over the...,0
...,...,...,...,...,...,...,...,...
4486,train_zero_shot.PT.351.8,PT,gato-pingado,zero_shot,Era o ano de 1968.,"A estação de passageiros tranquila, com um ou...","Com os primeiros raios solares, o potente DC 8...",0
4487,train_zero_shot.PT.351.9,PT,gato-pingado,zero_shot,De acordo com informações vazadas de dentro da...,"Segundo a fonte, o programa vai seguir um form...","Leonardo fica inchado, perde a voz e vício mor...",0
4488,train_zero_shot.PT.351.10,PT,gato-pingado,zero_shot,O radialista Roberto Toledo apresentou o show ...,"""Eu fiquei sentado no chão e realmente só tinh...",Entre os poucos espectadores estava o advogado...,0
4489,train_zero_shot.PT.351.11,PT,gato-pingado,zero_shot,O carnaval em Goiânia também tem espaço para o...,"É o caso do bloco “Gato Pingado”, que reuniu c...","Segundo os organizadores, quando criaram o blo...",1


In [15]:
df_one = pd.read_csv("/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_one_shot.csv")
df_one

,DataID,Language,MWE,Setting,Previous,Target,Next,Label
0,train_one_shot.EN.147.1,EN,high life,one_shot,PSG ace Marquinhos owns this £170k Ferrari - b...,Despite having the riches to afford the high l...,In an interview with the French magazine 'So F...,1
1,train_one_shot.EN.183.1,EN,birth rate,one_shot,Demography expert Piotr Szukalski told Dzienni...,Minister of Family and Social Policy Marlena M...,Commenting on data the state agency Statistics...,1
2,train_one_shot.EN.213.1,EN,home run,one_shot,Negro Leagues Baseball Museum President Bob Ke...,So Aaron faced the same brutal racism other Bl...,This Black man in the deep South was about to ...,0
3,train_one_shot.EN.26.1,EN,public service,one_shot,The Nakayama Public Service Scholarship is par...,Program leaders said the scholarship defines p...,I think everyone can agree we need to do more ...,0
4,train_one_shot.EN.26.2,EN,public service,one_shot,He was absolutely right.”,"In the ensuing years, Wennberg might not have ...",Wennberg said he knew when he ran for his sixt...,1
...,...,...,...,...,...,...,...,...
135,train_one_shot.PT.293.1,PT,amigo oculto,one_shot,"Mesmo com a pandemia, é possível manter a trad...",Chegou o fim do ano e com ele as festas de Nat...,"Não tem jeito, essa é uma tradição brasileira.",0
136,train_one_shot.PT.293.2,PT,amigo oculto,one_shot,Secret Santa Generator : Secret Santa é o nome...,Esta é a versão do app semelhante ao Amigo Ocu...,Eles são análogos e funcionam exatamente da me...,1
137,train_one_shot.PT.284.1,PT,agente secreto,one_shot,"Já à meia-noite, o Domingo Maior exibe 3 Dias ...","Ethan Renner (Kevin Costner), veterano agente ...","Seu último desejo é reatar com sua filha, com ...",1
138,train_one_shot.PT.435.1,PT,sangue frio,one_shot,"Com a pele em chamas, o companheiro de Khen (P...","Irritado com a mentira, o líder desejará matar...","Assim, ele decidirá se livrar do escolhido e l...",0


In [16]:
outpath = 'Data'
    
Path( os.path.join( outpath, 'ZeroShot' ) ).mkdir(parents=True, exist_ok=True)
Path( os.path.join( outpath, 'OneShot' ) ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath )

Wrote Data/ZeroShot/train.csv
Wrote Data/ZeroShot/dev.csv
Wrote Data/ZeroShot/eval.csv
Wrote Data/OneShot/train.csv
Wrote Data/OneShot/dev.csv
Wrote Data/OneShot/eval.csv


In [17]:
# model_config = BartConfig.from_pretrained(pretrain_model_dir) model_config.num_labels=new_num_labels model = BartForSequenceClassification.from_pretrained(pretrain_model_dir, config=model_config)

In [24]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'xlm-roberta-large' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 16 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

12/11/2021 14:14:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/11/2021 14:14:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0

In [25]:
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path '/content/models/ZeroShot/0' \
    	--do_predict \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 9 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/eval-dev/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
      --test_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

12/11/2021 14:59:22 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/11/2021 14:59:22 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=False,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0

In [26]:
params = {
    'submission_format_file' : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_submission_format.csv' ,
    'input_file'             : '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev.csv'                   ,
    'prediction_format_file' : '/content/models/ZeroShot/0/eval-dev/test_results_None.txt'                        ,
    }
params[ 'setting' ] = 'zero_shot'

In [27]:
 updated_data = insert_to_submission_file( **params )

In [28]:
!mkdir -p outputs

In [29]:
write_csv( updated_data, 'outputs/zero_shot_dev_formated.csv' ) 

Wrote outputs/zero_shot_dev_formated.csv


In [30]:
import sys
sys.path.append( '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/' ) 
from SubTask1Evaluator import evaluate_submission


submission_file = 'outputs/zero_shot_dev_formated.csv'
gold_file       = '/content/SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold.csv'

results = evaluate_submission( submission_file, gold_file )
%reload_ext google.colab.data_table
import pandas as pd
df = pd.DataFrame(data=results[1:], columns=results[0])
df

,Settings,Languages,F1 Score (Macro)
0,zero_shot,EN,0.761833
1,zero_shot,PT,0.646493
2,zero_shot,"EN,PT",0.734008
3,one_shot,EN,"(None, None, None)"
4,one_shot,PT,"(None, None, None)"
5,one_shot,"EN,PT","(None, None, None)"
